A script for scraping comments from news videos on youtube.com

In [1]:
import argparse
import re
from multiprocessing.pool import ThreadPool as Pool
import requests
import bs4 

In [2]:
root_url = 'http://www.youtube.com'
index_url = root_url + '/results?search_query=war+in+ukraine+latest'

In [3]:
def get_video_page_urls():
    response = requests.get(index_url)
    soup = bs4.BeautifulSoup(response.text)
    return [a.attrs.get('href') for a in soup.select('div.yt-lockup-content a[href^=/watch]')]

In [4]:
def get_video_data(video_page_url):
    video_data = {}
    response = requests.get(root_url + video_page_url.replace('watch','all_comments'))
    soup = bs4.BeautifulSoup(response.text)
    tmp=soup.find(id='watch-response-header-content')
    tmp2=re.split(unichr(8226),tmp.p.get_text())
    
    video_data['title'] = tmp.a.get_text()
    video_data['author'] = re.sub(' ','',re.sub('by ','',tmp2[0]))
    video_data['views'] = int(re.sub(',','',re.sub(' views', '',tmp2[1])))
    video_data['comments'] = int(re.search('\(([0-9]+)\)', soup.select('div.all-comments a')[0].get_text()).group(1))
    
    return video_data 

In [5]:
def show_video_stats():
    
    pool = Pool(4)
    video_page_urls = get_video_page_urls()
    results = sorted(pool.map(get_video_data, video_page_urls), key=lambda video: video['views'], reverse=True)
    max=len(results)
    print max
            
    print ('%-*s %-*s %-*s %s' % (60,'Title', 20, 'Author', 10, 'Views', 'Comments'))
    for i in range(max):
        s1=(results[i]['title'][:58] + '..') if len(results[i]['title']) > 60 else results[i]['title']
        s2=(results[i]['author'][:18] + '..') if len(results[i]['author']) > 20 else results[i]['author']
        print ('%-*s %-*s %-*u %u' % (60,s1, 20, s2, 10, results[i]['views'], results[i]['comments']))

In [6]:
show_video_stats()

20
Title                                                        Author               Views      Comments
War in Ukraine  Bloody battles for the South East / Война .. УкраинавОгне         604608     869
Ukraine War - Heavy Combat Action During Fighting Between .. WarLeaks             132091     987
Ukraine WAR: Video Battle Shooting at Donetsk, Ukraine | F.. TodayWorldNews       23699      13
12.12.2014 Ukrainian crisis news. Latest news of Ukraine, .. SOUTHFRONT           20322      65
16.01.2015 Ukrainian crisis news. Epic Urkainian victories.. SOUTHFRONT           18518      79
Ukraine War - Newest-Russian only ASVK counter-sniper and .. PressTVNews          15594      75
Ukraine War 2015 - Heavy Clashes And Fighting In Eastern U.. WarLeaks             15149      182
Ukraine War - Ukrainian Military Train. Donetsk region - U.. PressTVNews          14682      54
19.01.2015 Ukrainian crisis news. Peski, Poroshenko, IAmVo.. SOUTHFRONT           12860      60
22.01.2015 Ukrainian crisis 